# Export Beta Job Name

## DevOps Account

* acc: mlops-devops
* IAM: mlops-devops-admin



## Modify CodeBuild Project

* codebuild: beta-training-scikit-bring-your-own
* change the buildspec.yml

## Add Default Environment Variable

* key: beta_data_location
* value: s3://beta-sagemaker-repo-us-west-2/iris/input/iris.csv

## buildsepc.yml

In [ ]:
version: 0.2

env:
  variables:
     beta_data_location: "beta account data location"
  secrets-manager:
     beta_access_key: beta/sagemaker:beta-sagemaker-access
     beta_secret_key: beta/sagemaker:beta-sagemaker-secret
  exported-variables:
     - job_name

phases:
  install:
    runtime-versions:
      python: 3.8
  build:
    commands:
      - pip install sagemaker
      - python trigger-beta-training.py $beta_access_key $beta_secret_key $beta_data_location
  post_build:
    commands:
      - job_name=$(cat output.json | jq -r '.job_name')

## Developers Account

* acc: mlops-dev
* IAM: mlops-devops-developers
* cloud9: mlops-works

# Modify trigger-beta-training

* cd trigger-beta-training
* git pull
* change trigger-beta-training.py
* git diff
* git add .
* git commit -m "export job_name"
* git push origin master

## trigger-beta-training.py

In [ ]:
import time, boto3, sys
import sagemaker as sage 
 
# Create unique job name 
job_name_prefix = 'scikit-bring-your-own' 
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime()) 
job_name = job_name_prefix + timestamp 
 
beta_access_key = sys.argv[1]
beta_secrete_key = sys.argv[2]
beta_data_location = sys.argv[3]

#devops
region_devops = boto3.Session().region_name
account_devops = boto3.client('sts').get_caller_identity()['Account']

# beta
ss_beta = boto3.Session(aws_access_key_id=beta_access_key, aws_secret_access_key=beta_secrete_key) 
sm_beta = sage.Session(boto_session=ss_beta) 
account_beta = ss_beta.client('sts').get_caller_identity()['Account'] 

image = '{}.dkr.ecr.{}.amazonaws.com/scikit_bring_your_own:latest'.format(account_devops, region_devops) 
role = 'arn:aws:iam::{}:role/SageMakerExecutionRole'.format(account_beta) 
data_location = beta_data_location
output_location = 's3://beta-sagemaker-repo-us-west-2/iris/output' 

tree = sage.estimator.Estimator(image, 
    role, 1, 'ml.c4.2xlarge', 
    output_path=output_location,
    sagemaker_session=sm_beta) 
tree.fit(inputs=data_location, job_name=job_name, wait=False)

with open('output.json', 'w+') as f:
    print(f'{{"job_name":"{job_name}"}}', file=f)

## DevOps Account

* acc: mlops-devops
* IAM: mlops-devops-admin

## Check the pipeline

![](./img/29.png)

## Release successfully

check the beta-training-scikit-bring-your-own coebuild job. You can see the job name export

![](./img/30.png)


# Developers Account

* acc: mlops-dev
* IAM: mlops-devops-developers
* cloud9: mlops-works

## Modify the Machine Learning Container

* cd ~/environment/scikit_bring_your_own/container/local_test
* rm test_dir/output/*
* git status
* git add .
* git commit -m "remove test_dir/output"
* git push origin master

## DevOps Account Pipeline release

* acc: mlops-devops
* IAM: mlops-devops-admin
* See the pipeline release for the remote training